In [1]:
import numpy as np
import matplotlib.pyplot as plt
from utils_3 import *

In [2]:
def dict_to_vector(params):
    keys = []
    count = 0
    for key in ["W1", "b1", "W2", "b2", "W3", "b3"]:
        new_vector = np.reshape(params[key], (-1, 1))
        keys = keys + [key]*new_vector.shape[0]
        
        if count == 0:
            theta = new_vector
        else:
            theta = np.concenate((theta, new_vector), axis=0)
        count += 1
    
    return theta, keys

def vector_to_dict(theta):
    params = {}
    params["W1"] = theta[:20].reshape((5, 4))
    params["b1"] = theta[20:25].reshape((5, 1))
    params["W2"] = theta[25:40].reshape((3, 5))  
    params["b2"] = theta[40:43].reshape((3, 1))
    params["W3"] = theta[43:46].reshape((1, 3))    
    params["b3"] = theta[46:47].reshape((1, 1))
    
    return params

def grads_to_vector(grads):
    count = 0
    for key in ["dW1", "db1", "dW2", "db2", "dW3", "db3"]:
        new_vector = np.reshape(grads[key], (-1, 1))
        
        if count == 0:
            theta = new_vector
        else:
            theta = np.concatenate((theta, new_vector), axis=0)
    return theta

$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

In [3]:
def forward_propagation(x, theta):
    J = theta * x
    return J

In [4]:
def backprop(x, theta):
    dtheta = forward_propagation(x, theta) / theta
    return dtheta

In [5]:
def gradient_check(x, theta, epsilon=1e-7):
    thetaplus = theta + epsilon
    thetaminus = theta - epsilon
    J_plus = forward_propagation(x, thetaplus)
    J_minus = forward_propagation(x, thetaminus)
    gradapprox = (J_plus - J_minus) / (2*epsilon)
    
    grad = backprop(x, theta)
    
    numerator = np.linalg.norm(grad - gradapprox)
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    
    difference = numerator / denominator
    
    if difference < 1e-7:
        print("The gradient is correct!")
    else:
        print("The gradient is wrong!")
    
    return difference

In [6]:
x, theta = 2, 4
difference = gradient_check(x, theta)
print("difference = " + str(difference))

The gradient is correct!
difference = 2.919335883291695e-10


# N-dimensional gradient check

In [8]:
def forward_propagation(x, Y, params):
    m = X.shape[1]
    
    W1 = params["W1"]
    b1 = params["b1"]
    W2 = params["W2"]
    b1 = params["b2"]
    W3 = params["W3"]
    b3 = params["b3"]
    
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)
    
    logprobs = np.multiply(Y, np.log(A3)) + np.multiply(1 - A3, np.log(1 - Y))
    cost = (-1/m) * logprobs
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

In [9]:
def backprop(X, Y, cache):
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = (1/m) * np.dot(dZ3, A2.T)
    db1 = (1/m) * np.sum(dZ3, axis=1, keepdims=True)
    
    dA2 = np.dot(W3.T, dZ2)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = (1/m) * np.dot(dZ1, X.T)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)
    
    grads = {
        "dZ3":dZ3, "dW3":dW3, "db3":db3,
        "dZ2":dZ2, "dA2":dA2, "dW2":dW2, "db2":db2,
        "dZ1":dZ1, "dA1":dA1, "dW1":dW1, "db1":db1
    }
    
    return grads
    

In [10]:
def gradient_check(params, grads, X, Y, epsilon = 1e-7):
    params_vectors, _ = dict_to_vector(params)
    grads = grads_to_vector(grads)
    num_params = params_vectors.shape[0]
    J_plus = np.zeros((num_params, 1))
    J_minus = np.zeros((num_params, 1))    
    gradapprox = np.zeros((num_params, 1))
    
    for i in range(num_params):
        thetaplus = np.copy(params_vectors)
        thetaplus[i, 0] = thetaplus[i, 0] + epsilon
        J_plus[i], _ = forward_propagation(X, Y, vector_to_dict(thetaplus))
        
        thetaminus = np.copy(params)
        thetaminus[i, 0] = thetaminus[i, 0] - epsilon
        J_minus[i], _ = forward_propagation(X, Y, vector_to_dict(thetaminus))
        
        gradapprox[i] = (J_plus - J_minus) / 2*epsilon
        numerator = np.linalg.norm(grads[i] - gradapprox[i])
        denominator = np.linalg.norm(grads[i]) + np.linalg.norm(gradapprox[i])
        difference = numerator / denumerator
        
        if difference > 2e-7:
            print ("There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
        else:
            print ("Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    return difference